In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib
import yfinance as yfin
import scipy 
import MH4518.WY.Utils.constants as cs
import MH4518.WY.Utils.Dates as dates 
import MH4518.WY.Utils.payoff as pf
import MH4518.WY.Utils.GBM as gbm
import MH4518.WY.Utils.ProductPrice as pp 

import warnings
warnings.filterwarnings("ignore")
import multiprocessing as mp
import pandas as pd
import logging


In [3]:
def getdata():
    data = yfin.download(['LONN.SW', 'SIKA.SW'], period= 'max')['Adj Close']
    data.index = data.index.tz_localize(None)
    data = data.loc[cs.initial_fixing_date: cs.final_fixing_date]
    return data 

data = getdata()

[*********************100%***********************]  2 of 2 completed


In [35]:
def read_hist_rates():
    swiss_1_week = pd.read_csv(r"C:\Users\Espietsp\PycharmProjects\Simulation Techniques\.venv\MH4518\WY\Data\Switzerland 1-Week Bond Yield Historical Data.csv", index_col = 0, parse_dates=True, infer_datetime_format=True)
    swiss_1_month = pd.read_csv(r"C:\Users\Espietsp\PycharmProjects\Simulation Techniques\.venv\MH4518\WY\Data\Switzerland 1-Month Bond Yield Historical Data.csv", index_col = 0 , parse_dates=True, infer_datetime_format=True)
    swiss_2_month = pd.read_csv(r"C:\Users\Espietsp\PycharmProjects\Simulation Techniques\.venv\MH4518\WY\Data\Switzerland 2-Month Bond Yield Historical Data.csv", index_col = 0 , parse_dates=True, infer_datetime_format=True)
    swiss_6_month = pd.read_csv(r"C:\Users\Espietsp\PycharmProjects\Simulation Techniques\.venv\MH4518\WY\Data\Switzerland 6-Month Bond Yield Historical Data.csv", index_col = 0 , parse_dates=True, infer_datetime_format=True)
    swiss_1_year = pd.read_csv(r"C:\Users\Espietsp\PycharmProjects\Simulation Techniques\.venv\MH4518\WY\Data\Switzerland 1-Year Bond Yield Historical Data.csv" , index_col = 0 , parse_dates=True, infer_datetime_format=True)
    combined = pd.concat([swiss_1_week[["Price"]], swiss_1_month[["Price"]], swiss_2_month[["Price"]], swiss_6_month[["Price"]], swiss_1_year[["Price"]]], axis = 1)
    combined.columns = ["1 week Price", "1 month Price", "2 month Price", "6 month Price", "1 Year Price"]
   
    return combined

df = read_hist_rates()
df


,1 week Price,1 month Price,2 month Price,6 month Price,1 Year Price
Date,,,,,
2022-04-01,-0.77,-0.80,-0.71,-0.70,-0.57
2022-04-02,-0.77,-0.90,-0.74,-0.70,NaN
2022-04-03,-1.00,-0.80,-0.80,-0.70,-0.57
2022-04-04,-0.89,-0.90,-0.71,-0.70,-0.55
2022-04-05,-0.95,-0.90,-0.71,-0.70,-0.55
...,...,...,...,...,...
2024-10-28,0.77,0.69,0.60,0.51,0.56
2024-10-29,0.85,0.77,0.80,0.51,0.61
2024-10-30,0.77,0.68,0.80,0.51,0.62


In [23]:
# Define a mapping from maturity labels to years
maturity_to_years = {
    '1 week Price': 1 / 52,
    '1 month Price': 1 / 12,
    '2 month Price': 2 / 12,
    '6 month Price': 6 / 12,
    '1 Year Price': 1
}

# Create a list of maturities and their corresponding years
maturities = df.columns.tolist()
times_to_maturity = np.array([maturity_to_years[m] for m in maturities])
times_to_maturity

array([0.01923077, 0.08333333, 0.16666667, 0.5       , 1.        ])

In [24]:
from scipy.optimize import minimize

def vasicek_zero_coupon_price(r_t, a, b, sigma, T, t=0):
    B = (1 - np.exp(-a * (T - t))) / a
    A = np.exp(
        (b - (sigma ** 2) / (2 * a ** 2)) * (B - (T - t)) -
        (sigma ** 2) * B ** 2 / (4 * a)
    )
    P = A * np.exp(-B * r_t)
    return P


In [25]:
def calibration_objective(params, r0, times_to_maturity, market_prices):
    a, b, sigma = params
    model_prices = vasicek_zero_coupon_price(
        r_t=r0,
        a=a,
        b=b,
        sigma=sigma,
        T=times_to_maturity
    )
    # Calculate the squared differences
    squared_diffs = (market_prices - model_prices) ** 2
    # Sum the squared differences
    return np.sum(squared_diffs)


In [ ]:
calibration_objective()

In [32]:



for date in df.index:
    market_prices = df.loc[date].values
    print(market_prices)
    times = times_to_maturity
    # Handle missing data
    valid = ~np.isnan(market_prices)
    market_prices = market_prices[valid]
    times = times[valid]

    print(market_prices)
    print(times)

[-0.77 -0.8  -0.71 -0.7  -0.57]
[-0.77 -0.8  -0.71 -0.7  -0.57]
[0.01923077 0.08333333 0.16666667 0.5        1.        ]
[-0.77 -0.9  -0.74 -0.7    nan]
[-0.77 -0.9  -0.74 -0.7 ]
[0.01923077 0.08333333 0.16666667 0.5       ]
[-1.   -0.8  -0.8  -0.7  -0.57]
[-1.   -0.8  -0.8  -0.7  -0.57]
[0.01923077 0.08333333 0.16666667 0.5        1.        ]
[-0.89 -0.9  -0.71 -0.7  -0.55]
[-0.89 -0.9  -0.71 -0.7  -0.55]
[0.01923077 0.08333333 0.16666667 0.5        1.        ]
[-0.95 -0.9  -0.71 -0.7  -0.55]
[-0.95 -0.9  -0.71 -0.7  -0.55]
[0.01923077 0.08333333 0.16666667 0.5        1.        ]
[-0.93 -0.9  -0.72 -0.7  -0.55]
[-0.93 -0.9  -0.72 -0.7  -0.55]
[0.01923077 0.08333333 0.16666667 0.5        1.        ]
[-0.96 -0.9  -0.69 -0.7  -0.52]
[-0.96 -0.9  -0.69 -0.7  -0.52]
[0.01923077 0.08333333 0.16666667 0.5        1.        ]
[-0.77 -0.8  -0.68 -0.7  -0.54]
[-0.77 -0.8  -0.68 -0.7  -0.54]
[0.01923077 0.08333333 0.16666667 0.5        1.        ]
[-0.77 -0.67   nan -0.7  -0.52]
[-0.77 -0.67 -0.7

In [36]:
def model_calibration():
    params_df = pd.DataFrame(index=df.index, columns=['a', 'b', 'sigma', 'r0'])

    for date in df.index:
        market_prices = df.loc[date].values
        times = times_to_maturity
        # Handle missing data
        valid = ~np.isnan(market_prices)
        market_prices = market_prices[valid]
        times = times[valid]
        
        if len(market_prices) < 3:
            continue  # Not enough data to calibrate
        
        # Initial guesses for a, b, sigma
        initial_guess = [2, 0.02, 0.01]
        r0 = -np.log(market_prices[0]) / times[0]  # Initial short rate estimate
        
        # Minimize the objective function
        bounds = [(None, None), (None, None), (None, None)]  # Bounds for a, b, sigma
        result = minimize(
            calibration_objective,
            x0=initial_guess,
            args=(r0, times, market_prices),
            bounds=bounds,
            method='L-BFGS-B'
        )
        
        if result.success:
            a_calibrated, b_calibrated, sigma_calibrated = result.x
            params_df.loc[date] = [a_calibrated, b_calibrated, sigma_calibrated, r0]
        else:
            print(f"Calibration failed on date {date}")

In [37]:
termstructure = model_calibration()

Calibration failed on date 2022-04-01 00:00:00
Calibration failed on date 2022-04-02 00:00:00
Calibration failed on date 2022-04-03 00:00:00
Calibration failed on date 2022-04-04 00:00:00
Calibration failed on date 2022-04-05 00:00:00
Calibration failed on date 2022-04-06 00:00:00
Calibration failed on date 2022-04-07 00:00:00
Calibration failed on date 2022-04-08 00:00:00
Calibration failed on date 2022-04-09 00:00:00
Calibration failed on date 2022-04-11 00:00:00
Calibration failed on date 2022-04-12 00:00:00
Calibration failed on date 2022-04-13 00:00:00
Calibration failed on date 2022-04-14 00:00:00
Calibration failed on date 2022-04-15 00:00:00
Calibration failed on date 2022-04-16 00:00:00
Calibration failed on date 2022-04-17 00:00:00
Calibration failed on date 2022-04-18 00:00:00
Calibration failed on date 2022-04-19 00:00:00
Calibration failed on date 2022-04-20 00:00:00
Calibration failed on date 2022-04-21 00:00:00
Calibration failed on date 2022-04-22 00:00:00
Calibration f

KeyboardInterrupt: 